In [1]:
# -*- coding: utf-8 -*-
# notebooks/03_iterative_classification_experiment.ipynb
# Eksperymentalna klasyfikacja transakcji przed modyfikacją run_etl.py
# Autor: Paweł Żołądkiewicz, Senior BI Analyst
# Cel: Testowanie reguł klasyfikacji z przesuniętą regułą gratisów

import pandas as pd
import numpy as np
from datetime import datetime
import sys
import os

sys.path.append('../')
from run_etl import load_raw_excel, clean_raw

print("EKSPERYMENTALNA KLASYFIKACJA TRANSAKCJI")
print("=" * 60)

EKSPERYMENTALNA KLASYFIKACJA TRANSAKCJI


In [2]:
print("KROK 1: Ładowanie i czyszczenie danych...")

raw_data = load_raw_excel("../data/raw/online_retail_II.xlsx")
clean_data = clean_raw(raw_data)

print(f"Dane do klasyfikacji: {len(clean_data):,} rekordów")

KROK 1: Ładowanie i czyszczenie danych...
[2025-08-23 15:34:57] Ładowanie surowych danych z: ../data/raw/online_retail_II.xlsx
[2025-08-23 15:35:56] Wczytano 1,067,371 rekordów z 2 arkuszy
[2025-08-23 15:35:56] Walidacja obecnosci kolumn RAW...
[2025-08-23 15:35:56] Konwersja typów (Quantity, Price, InvoiceDate)...
[2025-08-23 15:35:58] Deduplikacja rekordów...
[2025-08-23 15:35:58] Usunięto duplikatów: 34,337
[2025-08-23 15:35:58] Filtrowanie prawidłowych sprzedaży...
[2025-08-23 15:35:58] Po filtrowaniu: 1,007,912 rekordów
[2025-08-23 15:35:58] Standaryzacja krajów...
[2025-08-23 15:35:58] Czyszczenie opisów produktu...
[2025-08-23 15:35:59] Kalkulacja Total_Value...
[2025-08-23 15:35:59] Łączny przychód: £20,476,082.17
[2025-08-23 15:35:59] Quality Gates po czyszczeniu...
Dane do klasyfikacji: 1,007,912 rekordów


In [7]:
print("KROK 2: Przygotowanie danych do klasyfikacji...")

# Skopiuj dane
df = clean_data.copy()

# Dodaj kolumny klasyfikacyjne
df['TransactionCategory'] = 'ToVerification'
df['ClassificationRule'] = 'None'
df['ClassificationStep'] = 0

print(f"Przygotowano {len(df):,} rekordów do klasyfikacji")

KROK 2: Przygotowanie danych do klasyfikacji...
Przygotowano 1,007,912 rekordów do klasyfikacji


In [8]:
def apply_rule(dataframe, rule_name , condition_func, category, step_number):
    """Aplikuje regułę klasyfikacji i zwraca statystyki"""
    mask = (dataframe['TransactionCategory'] == 'ToVerification') & dataframe.apply(condition_func, axis=1)
    count = mask.sum()

    if count > 0:
        dataframe.loc[mask, 'TransactionCategory'] = category
        dataframe.loc[mask, 'ClassificationRule'] = rule_name
        dataframe.loc[mask, 'ClassificationStep'] = step_number

    percentage = (count / len(dataframe)) * 100
    remaining = (dataframe['TransactionCategory'] == 'ToVerification').sum()
    remaining_pct = (remaining / len(dataframe)) * 100

    print(f" {rule_name}: {count:,} rekordów ({percentage:.1f}%)")

    if remaining_pct < 5.0:
        print(f"UWAGA: ToVerification spadło poniżej 5% - rozważ zakończenie iteracji")

    return count, remaining_pct

def show_classification_summary(dataframe):
    """Pokazuje podsumowanie klasyfikacji"""
    summary = dataframe['TransactionCategory'].value_counts().sort_values(ascending=False)
    print("\nPODSUMOWANIE KLASYFIKACJI:")
    for category, count in summary.items():
        percentage = (count / len(dataframe)) * 100
        print(f" {category}: {count:,} ({percentage:.1f}%)")

print("Funkcje pomocnicze zdefiniowane")

Funkcje pomocnicze zdefiniowane


In [9]:
print("REGUŁA 1: Opłaty pocztowe...")

def is_postage(row):
    stock_code = str(row['StockCode']).upper()
    description = str(row['Description']).upper()
    return (stock_code in ['POST', 'DOT'] or
            'POSTAGE' in stock_code or
            'POSTAGE' in description)

apply_rule(df, "Opłaty_Pocztowe", is_postage, "Postage", 1)

REGUŁA 1: Opłaty pocztowe...
 Opłaty_Pocztowe: 3,266 rekordów (0.3%)


(np.int64(3266), np.float64(99.67596377461524))

In [10]:
print("REGUŁA 2: Rabaty...")

def is_discount(row):
    stock_code = str(row['StockCode']).upper()
    description = str(row['Description']).upper()
    return (stock_code == 'D' or description == 'DISCOUNT')

apply_rule(df, "Rabaty", is_discount, "Discount", 2)

REGUŁA 2: Opłaty pocztowe...
 Rabaty: 5 rekordów (0.0%)


(np.int64(5), np.float64(99.67546769956107))

In [11]:
print("REGUŁA 3: Opłaty bankowe...")

def is_bank_charge(row):
    stock_code = str(row['StockCode']).upper()
    description = str(row['Description']).upper()
    return (stock_code == 'BANK CHARGES' or description == 'BANK CHARGES')

apply_rule(df, "Oplaty_Bankowe", is_bank_charge, "BankCharges", 3)

REGUŁA 3: Opłaty bankowe...
 Oplaty_Bankowe: 34 rekordów (0.0%)


(np.int64(34), np.float64(99.67209438919271))